In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from hyperopt import fmin, tpe, hp, anneal, Trials
from pathlib import Path
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge

import pickle

# 調完參數後，讀取history，找出最佳參數，再訓練成新模型，存成pickle
def get_optimal_parameter(history_file):
    optimal_parameter = pd.read_csv(history_file).sort_values('loss').iloc[0]
    return optimal_parameter.to_dict()


# 實際再訓練一次模型
def train_with_optimal_parameter(optimal_parameter):
    init = optimal_parameter["init"]
    min_samples_split = optimal_parameter["min_samples_split"]
    learning_rate = optimal_parameter["learning_rate"]
    max_depth = optimal_parameter["max_depth"]
    min_samples_leaf = optimal_parameter["min_samples_leaf"]
    n_estimators = optimal_parameter["n_estimators"]

    if init == "True":
        init_estimator = Ridge()
        init_estimator.fit(X_train, Y_train)

    else:
        init_estimator = None

    gbm = GradientBoostingRegressor(init=init_estimator,
                    min_samples_split=min_samples_split,
                    learning_rate=learning_rate,
                    max_depth=max_depth,
                    min_samples_leaf=min_samples_leaf,
                    max_features='sqrt',
                    loss='huber',
                    subsample=0.8,
                    random_state=50,
                    n_estimators=n_estimators,
                    verbose=1)


    gbm.fit(X_train, Y_train)
    return gbm

# 沒有調參數的模型
def train_without_optimal_parameter():
    init_estimator = Ridge()
    init_estimator.fit(X_train, Y_train)
    if county_name == '台北':
        gbm = GradientBoostingRegressor(min_samples_split=0.09, learning_rate=0.1, max_depth=7,
                                        min_samples_leaf=2,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=2000, verbose=1)
    elif county_name == '新北':
        gbm = GradientBoostingRegressor(init=init_estimator, min_samples_split=0.1, learning_rate=0.3, max_depth=12,
                                        min_samples_leaf=3,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=1600, verbose=1)
    elif county_name == '嘉義':
        gbm = GradientBoostingRegressor(min_samples_split=0.0109, learning_rate=0.03,
                                        max_depth=8, min_samples_leaf=5,  max_features='sqrt',
                                        subsample=0.8, random_state=50, n_estimators=600, verbose=1)
    elif county_name == '基隆':
        gbm = GradientBoostingRegressor(min_samples_split=0.1, learning_rate=0.05, max_depth=9,
                                        min_samples_leaf=5,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=1600, verbose=1)
    
    elif county_name == '桃園':
        gbm = GradientBoostingRegressor(init=init_estimator, min_samples_split=0.14, learning_rate=0.15, max_depth=11,
                                        min_samples_leaf=2,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=1800, verbose=1)
    elif county_name == '台中':
        gbm = GradientBoostingRegressor(init=init_estimator, min_samples_split=0.1, learning_rate=0.3, max_depth=10,
                                        min_samples_leaf=5,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=1400, verbose=1)
    elif county_name == '台南':
        gbm = GradientBoostingRegressor(init=init_estimator, min_samples_split=0.05, min_samples_leaf=5, max_depth=11,
                                        max_features='sqrt', subsample=0.8, random_state=50,
                                        learning_rate=0.02, n_estimators=1800, verbose=1)
    elif county_name == '高雄':
        gbm = GradientBoostingRegressor(init=init_estimator, min_samples_split=0.1, learning_rate=0.2, max_depth=9,
                                        min_samples_leaf=2,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=1600, verbose=1)
    elif county_name == '新竹':
        gbm = GradientBoostingRegressor(min_samples_split=0.1, learning_rate=0.15, max_depth=8,
                                        min_samples_leaf=4,  max_features='sqrt', loss='huber',
                                        subsample=0.8, random_state=50, n_estimators=1000, verbose=1)

    gbm.fit(X_train, Y_train)
    return gbm

def read_df(county_name):
    PATH = '.\\data'
    X_train = pd.read_csv(Path(PATH)/f"{county_name}_X_train.csv")
    Y_train = pd.read_csv(Path(PATH)/f"{county_name}_Y_train.csv")
    X_test = pd.read_csv(Path(PATH)/f"{county_name}_X_test.csv")
    Y_test = pd.read_csv(Path(PATH)/f"{county_name}_Y_test.csv")

    return X_train, Y_train, X_test, Y_test


if not os.path.exists('./optimal_models'):
    os.mkdir('./optimal_models')
if not os.path.exists('./non_optimal_models'):
    os.mkdir('./non_optimal_models')

county_name_list = ['台北','桃園','高雄','基隆','新竹','新北']
# county_name = '台北' 
for county_name in county_name_list:
    X_train, Y_train, X_test, Y_test = read_df(county_name)

    # 訓練前要去掉raw_time
    X_train = X_train.drop(columns=['raw_time'])
    X_test = X_test.drop(columns=['raw_time'])
    Y_train = Y_train.drop(columns=['raw_time'])
    Y_test = Y_test.drop(columns=['raw_time'])

    # 選擇要用的history file (原始版本 或 使用修改後的validation set版本)
    # history_file = Path('.\\history_tuning')/Path(f"{county_name}result.csv")
    history_file = Path('.\\history_tuning\\use_better_validation')/Path(f"{county_name}result.csv")
    optimal_parameter = get_optimal_parameter(history_file)

    # 做調參數模型
    optimal_model = train_with_optimal_parameter(optimal_parameter)
    with open(f'./optimal_models/{county_name}_model', 'wb') as f:
        pickle.dump(optimal_model, f)

    # 做不調參數模型
    non_optimal_model = train_without_optimal_parameter()
    with open(f'./non_optimal_models/{county_name}_model', 'wb') as f:
        pickle.dump(non_optimal_model, f)

In [22]:
# 讀pickle模型，預測
import joblib

county_name = '台北'
model = joblib.load(f'./optimal_models/{county_name}_model')
# 讀取未調參數版本的model
non_optimal_model = joblib.load(f'./non_optimal_models/{county_name}_model')
X_train, Y_train, X_test, Y_test = read_df(county_name)
# 訓練時沒有用raw_time欄位，但現在要測模型(X_train+X_test都測)所以也要拿掉raw_time
X_all_data = pd.concat([X_train, X_test])
raw_time = X_all_data['raw_time'].to_numpy()
X_all_data = X_all_data.drop(columns=['raw_time'])

Y_all_data = pd.concat([Y_train, Y_test])

result = model.predict(X_all_data)
non_optimal_result = non_optimal_model.predict(X_all_data)

# 加回去raw_time, 真實y
result_table = pd.DataFrame()
result_table['raw_time'] = raw_time
result_table['LRHP'] = Y_all_data.reset_index()['LRHP']

# 加回去預測結果
result_table['y_hat_tuned'] = result
result_table['y_hat_without_tuned'] = non_optimal_result

result_table['RHP'] = np.exp(result_table['LRHP'])
result_table['y_hat_tuned'] = np.exp(result_table['y_hat_tuned'])
result_table['y_hat_without_tuned'] = np.exp(result_table['y_hat_without_tuned'])

# 加回去hit rate
def hit_rate(cola, colb, threshold):
    # cola: 實際值
    # colb: 預測值
    # threshold: 0.1
    if abs((cola-colb) / cola) <= threshold:
        return 1
    else:
        return 0

result_table['1%_hit_rate_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_tuned'], 0.01), axis=1)
result_table['1%_hit_rate_without_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_without_tuned'], 0.01), axis=1)
result_table['5%_hit_rate_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_tuned'], 0.05), axis=1)
result_table['5%_hit_rate_without_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_without_tuned'], 0.05), axis=1) 
result_table['10%_hit_rate_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_tuned'], 0.1), axis=1)
result_table['10%_hit_rate_without_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_without_tuned'], 0.1), axis=1)
result_table['15%_hit_rate_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_tuned'], 0.15), axis=1)
result_table['15%_hit_rate_without_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_without_tuned'], 0.15), axis=1)
result_table['20%_hit_rate_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_tuned'], 0.2), axis=1)
result_table['20%_hit_rate_without_tuned'] = result_table.apply(lambda x: hit_rate(x['LRHP'], x['y_hat_without_tuned'], 0.2), axis=1)


In [24]:
# 產製結果
def compare_tuned_and_without_tuned_model(result_table):
    result = pd.DataFrame()
    result['MAPE_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: mean_absolute_percentage_error(x['LRHP'], x['y_hat_tuned'])
    )
    result['MAPE_without_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: mean_absolute_percentage_error(x['LRHP'], x['y_hat_without_tuned'])
    )
    result['1%_hit_rate_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['1%_hit_rate_tuned'].sum()/len(x['1%_hit_rate_tuned'])
    )
    result['1%_hit_rate_without_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['1%_hit_rate_without_tuned'].sum()/len(x['1%_hit_rate_without_tuned'])
    )
    result['5%_hit_rate_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['5%_hit_rate_tuned'].sum()/len(x['5%_hit_rate_tuned'])
    )
    result['5%_hit_rate_without_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['5%_hit_rate_without_tuned'].sum()/len(x['5%_hit_rate_without_tuned'])
    )
    result['10%_hit_rate_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['10%_hit_rate_tuned'].sum()/len(x['10%_hit_rate_tuned'])
    )
    result['10%_hit_rate_without_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['10%_hit_rate_without_tuned'].sum()/len(x['10%_hit_rate_without_tuned'])
    )
    result['15%_hit_rate_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['15%_hit_rate_tuned'].sum()/len(x['15%_hit_rate_tuned'])
    )
    result['15%_hit_rate_without_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['15%_hit_rate_without_tuned'].sum()/len(x['15%_hit_rate_without_tuned'])
    )
    result['20%_hit_rate_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['20%_hit_rate_tuned'].sum()/len(x['20%_hit_rate_tuned'])
    )
    result['20%_hit_rate_without_tuned'] = result_table.groupby('raw_time').apply(
        lambda x: x['20%_hit_rate_without_tuned'].sum()/len(x['20%_hit_rate_without_tuned'])
    )
    return result

In [27]:
result = compare_tuned_and_without_tuned_model(result_table)

if not os.path.exists('./compare_tuned_and_without_tuned_model'):
    os.mkdir('./compare_tuned_and_without_tuned_model')
result.to_csv(f'./compare_tuned_and_without_tuned_model/{county_name}.csv')

result

,MAPE_tuned,MAPE_without_tuned,1%_hit_rate_tuned,1%_hit_rate_without_tuned,5%_hit_rate_tuned,5%_hit_rate_without_tuned,10%_hit_rate_tuned,10%_hit_rate_without_tuned,15%_hit_rate_tuned,15%_hit_rate_without_tuned,20%_hit_rate_tuned,20%_hit_rate_without_tuned
raw_time,,,,,,,,,,,,
10108,0.004385,0.007799,0.915956,0.724726,1.000000,0.998782,1.0,1.0,1.0,1.0,1.0,1.0
10109,0.004236,0.007044,0.923690,0.763098,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
10110,0.004201,0.007388,0.921154,0.733654,0.999038,0.999038,1.0,1.0,1.0,1.0,1.0,1.0
10111,0.004527,0.007541,0.907522,0.738594,1.000000,0.998767,1.0,1.0,1.0,1.0,1.0,1.0
10112,0.004034,0.006958,0.929273,0.749509,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11108,0.006956,0.007449,0.753165,0.727848,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
11109,0.007085,0.007779,0.740260,0.746753,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
11110,0.007373,0.007769,0.722603,0.722603,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
